In [5]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
import IPython
from tensorflow import keras
import kerastuner as kt
import datetime
from tensorboard.plugins.hparams import api as hp

mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images.astype('float64') / 255.0

test_images = test_images.astype('float64') / 255.0

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

def model_builder(hp):
    global model
    model = keras.Sequential()

    global hp_units
    hp_units = hp.Int('units', min_value=2, max_value=99999, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))

    f = hp.Choice('False', 'True')

    global hp_dropout

    hp_dropout = 0

    if f == 'False':
      hp_dropout = hp.Choice('dropout', values=(0.1, 0.2,0.3,0.4,0.5, 0.6, 0.7, 0.8, 0.9))
    if f == 'True':
      hp_dropout = 0
      
    model.add(keras.layers.Dropout(hp_dropout))
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    global hp_learning_rate
    hp_learning_rate = hp.Choice('learning_rate', values=(1e-5, 1e-4, 1e-6, 1e-7, 1e-1, 1e-2, 1e-3, 1e-4))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model


tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='Fashion_mnist_KT_logs')

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()


class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)


log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
optimizer = tf.keras.optimizers.Adam(hp_learning_rate)

tuner.search(train_images, train_labels, epochs=15, validation_data=(test_images, test_labels),
             callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyper-parameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}
""")

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/fit/' + current_time + '/train'
test_log_dir = 'logs/fit/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
logdir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model.fit(train_images, train_labels, epochs=10, verbose=0,
          callbacks=[tensorboard_callback])

model.trainable = False

model.summary()

train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

def run(run_dir, hp):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hp)  # record the values used in this trial
        accuracy = model_builder(hp)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


def train_step(model, optimizer, train_images, train_labels):
    with tf.GradientTape() as tape:
        predictions = model(train_images, training=True)
        loss = loss_object(train_labels, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(train_labels, predictions)


def test_step(model, test_images, test_labels):
    predictions = model(test_images)
    loss = loss_object(test_labels, predictions)

    test_loss(loss)
    test_accuracy(test_labels, predictions)

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

for epoch in range(15):
    for (train_images, train_labels) in train_dataset:
        train_step(model, optimizer, train_images, train_labels)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (test_images, test_labels) in test_dataset:
        test_step(model, test_images, test_labels)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

print('\nTest accuracy:', test_acc)

probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])

predictions = probability_model.predict(test_images)


def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                         100 * np.max(predictions_array),
                                         class_names[true_label]),
               color=color)


def plot_value_array(i, predictions_array, true_label):
    true_label = true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')


num_rows = 3
num_cols = 9
num_images = num_rows * num_cols
plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()


INFO:tensorflow:Reloading Oracle from existing project my_dir/Fashion_mnist_KT_logs/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/Fashion_mnist_KT_logs/tuner0.json
INFO:tensorflow:Oracle triggered exit

The hyper-parameter search is complete. The optimal number of units in the first densely-connected
layer is 66 and the optimal learning rate for the optimizer
is 0.1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (32, 28, 2)               58        
_________________________________________________________________
dropout (Dropout)            (32, 28, 2)               0         
_________________________________________________________________
dense_1 (Dense)              (32, 28, 10)              30        
_________________________________________________________________
flatten (Flatten)            (32, 280)                 0         
Total 

TypeError: ignored

In [2]:
pip -q install keras-tuner

     |████████████████████████████████| 71kB 8.8MB/s 
